Fase 3 – Fairness & Bias Evaluation

In [34]:


# --- 1. Imports and data ---
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from fairlearn.metrics import MetricFrame, selection_rate

# Load dataset
df = pd.read_csv("data/raw/adult.csv").dropna(subset=["sex", "race", "income"])

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include="object"):
    df[col] = le.fit_transform(df[col])

# Split features and target
X = df.drop("income", axis=1)
y = df["income"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- 2. Train model ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(solver="saga", max_iter=5000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# --- 3. Fairness analysis ---
# Sensitive feature (gender)
sensitive_features = df.loc[X_test.index, "sex"]

# Create MetricFrame
mf = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "recall": recall_score,
        "selection_rate": selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_features
)

# Display results
print("📊 Metrics by gender group:")
mf_df = mf.by_group.rename(index={0: "Female", 1: "Male"})
print(mf_df)



📊 Metrics by gender group:
        accuracy   recall  selection_rate
sex                                      
Female  0.907850  0.20000        0.030717
Male    0.788573  0.49602        0.208983


In [35]:
# Base rate (verdadeiro) por grupo
gt_rate = (
    pd.DataFrame({
        "sex": df.loc[X_test.index, "sex"],
        "y_true": y_test
    })
    .groupby("sex")["y_true"]
    .mean()
    .rename("ground_truth_positive_rate")
)
print(gt_rate)  # ex.: Female 0.06 vs Male 0.20


sex
0    0.102389
1    0.307058
Name: ground_truth_positive_rate, dtype: float64


In [36]:
from sklearn.metrics import recall_score, confusion_matrix

# Separar por grupo
mask_f = (df.loc[X_test.index, "sex"] == 0)  # Female
mask_m = (df.loc[X_test.index, "sex"] == 1)  # Male

# Recall (TPR) por grupo – entre as pessoas >50K, quantas o modelo acerta?
tpr_f = recall_score(y_test[mask_f], y_pred[mask_f])  # equal opportunity para Female
tpr_m = recall_score(y_test[mask_m], y_pred[mask_m])  # equal opportunity para Male
print("TPR/Recall  Female:", round(tpr_f, 3))
print("TPR/Recall  Male  :", round(tpr_m, 3))

# FPR por grupo – entre as pessoas ≤50K, quantas o modelo erra como >50K?
def fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return fp / (fp + tn)

fpr_f = fpr(y_test[mask_f], y_pred[mask_f])
fpr_m = fpr(y_test[mask_m], y_pred[mask_m])
print("FPR Female:", round(fpr_f, 3))
print("FPR Male  :", round(fpr_m, 3))


TPR/Recall  Female: 0.2
TPR/Recall  Male  : 0.496
FPR Female: 0.011
FPR Male  : 0.082


In [37]:
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

sensitive = df.loc[X_test.index, "sex"]
dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive)  # seleção
eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive)     # TPR & FPR
print("Demographic Parity Difference:", round(dp_diff, 3))
print("Equalized Odds Difference    :", round(eo_diff, 3))


Demographic Parity Difference: 0.178
Equalized Odds Difference    : 0.296


# The initial logistic regression model exhibited a gender bias,
# predicting significantly more high-income individuals among men than women.
# This imbalance reflected historical patterns in the dataset, where male samples
# were overrepresented in the >50K income category.
#
# To mitigate this bias, the Fairlearn ThresholdOptimizer was applied
# with the 'true_positive_rate_parity' (Equal Opportunity) constraint.
# This approach adjusted the decision thresholds separately for each gender group,
# ensuring that women and men had an equal chance (recall rate) of being correctly
# identified as high-income individuals.
#
# After applying the fairness mitigation, the recall rates between groups became
# nearly equal (around 0.36 for both genders), while overall model performance
# remained stable. This demonstrates a fairer and more responsible AI model
# that balances predictive accuracy with ethical accountability.

In [41]:
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.metrics import accuracy_score, recall_score

# Sensitive feature
sensitive = df.loc[X_test.index, "sex"]

# Pós-processamento usando o modelo diretamente
postproc = ThresholdOptimizer(
    estimator=model,
    constraints="true_positive_rate_parity",  # Equal Opportunity
    predict_method="predict_proba"
)

# Ajustar e prever
postproc.fit(X_test_scaled, y_test, sensitive_features=sensitive)
y_post = postproc.predict(X_test_scaled, sensitive_features=sensitive)

# Avaliar novamente
mf_post = MetricFrame(
    metrics={"accuracy": accuracy_score, "recall": recall_score, "selection_rate": selection_rate},
    y_true=y_test,
    y_pred=y_post,
    sensitive_features=sensitive
)

print("📊 After Fairness Mitigation:")
print(mf_post.by_group)


📊 After Fairness Mitigation:
     accuracy    recall  selection_rate
sex                                    
0    0.888613  0.357576        0.082222
1    0.777880  0.361194        0.136877
